In [1]:
import json
import pandas as pd

In [2]:
with open("./query.json", 'r') as f:
    data = json.load(f)['data']['swaps']
for d in data:
    d["transaction_blockNumber"] = d["transaction"]["blockNumber"]
    del d["transaction"]

In [3]:
data[0]

{'amount0In': '70.747260165322752132',
 'amount0Out': '0',
 'amount1In': '0',
 'amount1Out': '0.608917827698621792',
 'id': '0xd655a1e18631ab5d1ff0c089c79621eecb4487cdf42a43905490258db284ff0a-0',
 'pair': {'token0': {'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
   'symbol': 'UNI',
   'totalLiquidity': '2367369.009729215038093377',
   'tradeVolume': '764099171.532566210480943776',
   'tradeVolumeUSD': '6641705341.919281754977178013108196'},
  'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
   'symbol': 'WETH',
   'totalLiquidity': '742987.744995710631690356',
   'tradeVolume': '243278168.81885744295780905',
   'tradeVolumeUSD': '272010992073.2826765029025952385042'}},
 'timestamp': '1624379026',
 'transaction_blockNumber': '12685071'}

In [4]:
fields = [field for field in data[0].keys() if field != "pair"]
for token in data[0]["pair"].keys():
    fields.extend([f"{token}_{field}" for field in data[0]["pair"][token] if field not in ["id", "symbol"]])

In [5]:
rawdf = {field: [] for field in fields}
for d in data:
    for field in rawdf.keys():
        if "token" in field:
            token, token_field = field.split('_')
            rawdf[field].append(d["pair"][token][token_field])
        else:
            rawdf[field].append(d[field])
rawdf = pd.DataFrame.from_dict(rawdf)

In [6]:
symbols = [data[0]["pair"][f"token{i}"]["symbol"] for i in range(2)]

rename_cols = {}
for col in rawdf.columns:
    if "amount" in col:
        idx = int(col[col.find("amount") + 6])
        rename_cols[col] = col.replace(f"amount{idx}", f"{symbols[idx]}_")
    if "token" in col:
        idx = int(col[col.find("token") + 5])
        rename_cols[col] = col.replace(f"token{idx}", f"{symbols[idx]}")
rawdf = rawdf.rename(rename_cols, axis=1)

array(['2367369.009729215038093377'], dtype=object)